### **FACT ORDERS**

In [0]:
df = spark.sql("select * from databrickseteproject167_catalog.silver.orders")
df.display()

In [0]:
df_dimcus = spark.sql("select DimCustomerKey, customer_id as dim_customer_id from databrickseteproject167_catalog.gold.dimcustomers order by DimCustomerKey")
df_dimcus.display()

In [0]:
df_dimpro = spark.sql("select product_id as DimProductKey, product_id as dim_product_id from databrickseteproject167_catalog.gold.dimproducts order by DimProductKey")
df_dimpro.display()

In [0]:
df_fact = df.join(df_dimcus, df['customer_id'] == df_dimcus['dim_customer_id'],how='left').join(df_dimpro, df['product_id'] == df_dimpro['dim_product_id'],how='left')
df_fact.display()

In [0]:
df_fact_new = df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')
df_fact_new.display()

### **Upsert on Fact Table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databrickseteproject167_catalog.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "databrickseteproject167_catalog.gold.FactOrders")
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id AND trg.DimCustomerKey = src.DimCustomerKey AND trg.DimProductKey = src.DimProductKey")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    df_fact_new.write.format("delta")\
            .option("path","abfss://gold@databrickseteproject167.dfs.core.windows.net/FactOrders")\
            .saveAsTable("databrickseteproject167_catalog.gold.FactOrders")

In [0]:
%sql
select * from databrickseteproject167_catalog.gold.FactOrders